In [1]:
import os
from glob import glob
from pathlib import Path

import numpy as np
import pandas as pd

ModuleNotFoundError: No module named 'numpy'

In [2]:
a = 1

In [4]:
mother_path = Path('D:/Multi-modal project/')

### Data curation for ephys behavioral data

In [26]:
data_path = mother_path / 'analysis' / 'matlab' / 'behavior' / 'unimodal session' / 'overall performance'
VA = pd.read_excel(data_path / 'multimodal.xlsx')
V = pd.read_excel(data_path / 'visual only.xlsx')
A = pd.read_excel(data_path / 'auditory only.xlsx')
C = pd.read_excel(data_path / 'elemental visual.xlsx')

data = [VA, V, A, C]

In [27]:
data[0]

,Unnamed: 0,session 1,session 2,session 3,session 4,session 5,session 6
0,LE600,85,93,92,82,90,NaN
1,LE602,89,88,87,94,91,NaN
2,LE633,78,87,83,79,81,86.0
3,LE640,77,88,91,81,86,NaN
4,LE647,80,80,81,66,80,82.0
5,LE654,75,87,80,86,77,84.0
6,LE679,75,86,90,86,95,94.0
7,LE699,95,97,89,94,99,98.0


In [34]:
result = []
cond = ['Multimodal', 'Visual', 'Auditory', 'Control']
ss = [5, 5, 6, 5, 6, 6, 6, 6]

for r in range(8):
    for s in range(ss[r]):    
        for c in range(4):
            this_cond = data[c]
            this_result = {'RatID': f'rat{r+7}',
                           'Cond': cond[c],
                           'Session': s+1,
                           'Correctness': this_cond.iloc[r,s+1]/100
                          }
            result.append(this_result)

In [35]:
result = pd.DataFrame(result)

result.to_csv(mother_path / 'Behavioral data (E-phys).csv')

---

### Data curation for ephys neural data

In [58]:
cell_path = mother_path / 'analysis' / 'result' / 'zFR export' / '13-Jun-2023'
cell_list = os.listdir(cell_path)

save_path = mother_path / 'analysis' / 'result' / 'zFR export' / 'for sharing'

In [75]:
rat_list = ['600', '602', '633', '640', '647', '654', '679', '699']

a = []
for cell_num, cell_name in enumerate(cell_list):
    # get information about the cell
    cell_info = cell_name.split('-')
    cell_id, rat_id, session_id, region = cell_info[0], cell_info[1], cell_info[2], cell_info[5]
    bg = float(cell_info[8].replace(' mm)',''))
    
    if region == 'PER':
        df = pd.read_csv(cell_path/cell_name)
        
        df.rename(columns={'Type': 'Condition'}, inplace=True)
        df.loc[df['Condition']=='Elemental', 'Condition'] = 'Control'
        
        for i in range(90):
            df.rename(columns={f'Var{i+10}':f'FR_bin{i+1}'},inplace=True)
        
        save_name = f'{cell_id}_rat{rat_list.index(rat_id)+7}_session{int(session_id)}_PER.csv'
        df.to_csv(save_path / save_name, index=False)